In [1]:
pip install requests beautifulsoup4 pandas


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


Step 2: Set Up Telegram API
Create a Telegram Application:

Go to my.telegram.org.
Log in with your phone number.
Create a new application to get the api_id and api_hash.
Initialize the Telegram Client:

In [3]:
def scrape_telegram_channel(channel_url, max_messages=100):
    messages = []
    base_url = 'https://t.me/s/'
    url = f'{base_url}{channel_url}'

    while len(messages) < max_messages:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        message_blocks = soup.find_all('div', class_='tgme_widget_message')

        for block in message_blocks:
            if len(messages) >= max_messages:
                break

            date = block.find('time')['datetime']
            text = block.find('div', class_='tgme_widget_message_text')
            text = text.get_text(strip=True) if text else ''
            sender = block.find('a', class_='tgme_widget_message_user')
            sender = sender.get_text(strip=True) if sender else 'Unknown'

            messages.append({
                'date': date,
                'text': text,
                'sender': sender
            })

        next_button = soup.find('a', class_='tme_messages_more')
        if next_button and next_button['href']:
            next_url = next_button['href']
            if next_url.startswith('/'):
                url = f'{base_url}{next_url}'
            else:
                url = next_url
        else:
            break

        time.sleep(1)  # Sleep to avoid making too many requests too quickly

    return messages


In [4]:
channel_urls = [
    'DoctorsET', 'lobelia4cosmetics', 'yetenaweg', 'EAHCI'
]

all_messages = []

for channel in channel_urls:
    messages = scrape_telegram_channel(channel, max_messages=100)
    all_messages.extend(messages)


In [5]:
df = pd.DataFrame(all_messages)
df.to_csv('scraped_telegram_data.csv', index=False)


In [6]:
df = pd.DataFrame(all_messages)


In [7]:
df.head(10)

,date,text,sender
0,2022-04-28T05:46:38+00:00,ሃያት ሆስፒታል ሜዲካል ኮሌጅህክምና ለማግኘት አስበዋል ?የት መሄድስ እን...,Unknown
1,2022-04-29T18:57:35+00:00,,Unknown
2,2022-04-29T18:59:05+00:00,,Unknown
3,2022-05-01T11:13:33+00:00,ከሳምንት በፊት በለቀቅነው ቦርጭን በህክምና እንዴት ማጥፋት ይቻላል በሚለ...,Unknown
4,2022-05-06T17:51:05+00:00,ዛሬ ምሽት 3:30 በ ፋና ቴሌቪዥንስለ ፕሮቲን : ክሬቲን ፓውደሮች ምን ...,Unknown
5,2022-05-07T18:22:14+00:00,በብዙ ማህበራዊ ሚዲያዎች በ ከፋተኛ ሁኔታ እየተሸጠ ያለውፕሮቲን- ክሬቲን...,Unknown
6,2022-05-15T15:59:10+00:00,ልጆች ላላችሁ / አባት እናት ወይንም እህት ወንድም / በቅርቡ ልጅ ለመው...,Unknown
7,2022-05-20T18:04:53+00:00,ከትንሽ ግዚያት በፊት ስፖርት መስራት እንደ ቅንጦት ይታይ ነበር አሁን ላ...,Unknown
8,2022-05-31T17:51:13+00:00,ድንገተኛ አደጋ / የአጥንት ስብራትአያርገውና ድንገተኛ የሆነ አደጋ ቢደር...,Unknown
9,2022-06-12T17:15:47+00:00,ስፖርት የመስራት ሱስ ይኖር ይሆን?በአሁኑ ወቅት ብዙ የስፖርት መስሪያ ቦ...,Unknown


In [8]:
df.shape

(400, 3)

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import logging
import re
from datetime import datetime
import time


In [10]:
logging.basicConfig(
    filename='image_scraping.log',
    filemode='a',
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)


In [16]:
def scrape_images_from_channel(channel_url, max_images=100):
    base_url = 'https://t.me/s/'
    url = f'{base_url}{channel_url}'
    images = []
    image_count = 0

    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        logger.error(f'HTTP error occurred while accessing {url}: {http_err}')
        return images

    soup = BeautifulSoup(response.content, 'html.parser')
    message_blocks = soup.find_all('div', class_='tgme_widget_message')

    for block in message_blocks:
        if image_count >= max_images:
            break

        image_tag = block.find('a', class_='tgme_widget_message_photo_wrap')
        if image_tag and 'style' in image_tag.attrs:
            style = image_tag['style']
            image_url = style.split('url(')[1].split(')')[0].replace("'", "")
            date_str = block.find('time')['datetime']
            date = datetime.strptime(re.sub(r'\+\d+:\d+$', '', date_str), '%Y-%m-%dT%H:%M:%S')
            sender = block.find('a', class_='tgme_widget_message_user')
            sender = sender.get_text(strip=True) if sender else 'Unknown'

            images.append({
                'date': date,
                'image_url': image_url,
                'sender': sender
            })

            image_count += 1

    return images


In [17]:
def download_image(image_url, save_path):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                file.write(response.content)
        else:
            logger.error(f'Failed to download image from {image_url}')
    except Exception as e:
        logger.error(f'Error downloading image {image_url}: {e}')


In [18]:
# Replace with actual channels that exist and have images
channel_urls = [
    'lobelia4cosmetics',  'Chemed Telegram Channe'
]

if not os.path.exists('images'):
    os.makedirs('images')

all_images = []

for channel in channel_urls:
    try:
        images = scrape_images_from_channel(channel, max_images=100)
        all_images.extend(images)

        # Save images locally
        for img in images:
            image_url = img['image_url']
            date = img['date']
            date_str = date.strftime('%Y%m%d_%H%M%S')
            save_path = f'images/{channel}_{date_str}.jpg'
            download_image(image_url, save_path)
            img['local_path'] = save_path
    except Exception as e:
        logger.error(f'Error scraping channel {channel}: {e}')

# Save metadata to CSV
df = pd.DataFrame(all_images)
df.to_csv('image_metadata.csv', index=False)


In [19]:
df_im=pd.read_csv('image_metadata.csv')
df_im.head()

,date,image_url,sender,local_path
0,2024-06-11 12:33:25,https://cdn4.cdn-telegram.org/file/AI9Z8g3JTfw...,Unknown,images/lobelia4cosmetics_20240611_123325.jpg
1,2024-06-11 12:33:25,https://cdn4.cdn-telegram.org/file/HpyI8Axk2cW...,Unknown,images/lobelia4cosmetics_20240611_123325.jpg
2,2024-06-11 12:33:25,https://cdn4.cdn-telegram.org/file/lJH6QEKaWPV...,Unknown,images/lobelia4cosmetics_20240611_123325.jpg
3,2024-06-11 12:33:25,https://cdn4.cdn-telegram.org/file/BoI4zqx7eLW...,Unknown,images/lobelia4cosmetics_20240611_123325.jpg
4,2024-06-11 12:33:25,https://cdn4.cdn-telegram.org/file/EgrHY_G_4AZ...,Unknown,images/lobelia4cosmetics_20240611_123325.jpg


In [20]:
df_im['image_url']

0     https://cdn4.cdn-telegram.org/file/AI9Z8g3JTfw...
1     https://cdn4.cdn-telegram.org/file/HpyI8Axk2cW...
2     https://cdn4.cdn-telegram.org/file/lJH6QEKaWPV...
3     https://cdn4.cdn-telegram.org/file/BoI4zqx7eLW...
4     https://cdn4.cdn-telegram.org/file/EgrHY_G_4AZ...
5     https://cdn4.cdn-telegram.org/file/n9qP0G3K0Os...
6     https://cdn4.cdn-telegram.org/file/rSLkk5cPL4H...
7     https://cdn4.cdn-telegram.org/file/oS8Hu4SASJ5...
8     https://cdn4.cdn-telegram.org/file/uatMUyfYn0P...
9     https://cdn4.cdn-telegram.org/file/M0zLzLrogpg...
10    https://cdn4.cdn-telegram.org/file/iPMlOcClEMj...
11    https://cdn4.cdn-telegram.org/file/sz9awauvRMi...
12    https://cdn4.cdn-telegram.org/file/wBqaHBIkuDm...
13    https://cdn4.cdn-telegram.org/file/rXhYAqjc2Cr...
14    https://cdn4.cdn-telegram.org/file/lC0UD8Q8F40...
15    https://cdn4.cdn-telegram.org/file/rUVvee2yvay...
16    https://cdn4.cdn-telegram.org/file/B_4ZUwbggc2...
17    https://cdn4.cdn-telegram.org/file/lIyy4Gw